<h1>Trees, Forests, and Democracy</h1>
<p>
    A new class of models will be featured in this notebook. But before we dive into it, lets sum up what we saw until now :
    <ul>
        <li>A Data Science project <em>always</em> starts with an <em>exploratory</em> phase</li>
        <li>Given a target, we can produce simple models that will establish correlations between what we aim to predict and <em>features</em> </li>
        <li>Any <em>categorical</em> feature can be (and must be) turned into numerical values.</li>
        <li> Also, new features can be created by combining other ones</li>
        <li>A model can be used to perform classification or regression depending on whether the target is categorical (or binary as we saw it) or continuous (as it was the case in the third notebook)</li>
    </ul>
</p>
<h2>Classification Trees </h2>
<p>
    The following cells will introduce you a new type of models : the classification trees. They work by learning simple rules on the features that will cut the dataset into smaller subsets</p>
    <img src="https://www.ibm.com/support/knowledgecenter/SS3RA7_15.0.0/com.ibm.spss.modeler.help/images/dectree.gif" />
<p>
    The <em>parameters</em> of a such model are the rules of each node, and the <em>shape</em> of the tree. To classify a sample, the model will simply make it flow through each node until it reaches a leaf.
</p>
<p>
    The learning process works by fiding <em>optimal</em> cuts. A cut is a split over a feature depending on a certain value : <code>price_first_item_purchased</code> > 125\$ for exemple is a cut.
    <br >
    To find those cuts, the algorithm uses a mesure called the <em>Gini impurty</em> : it represents the probability of miss-classifying a sample based on a certain cut. The <a href="https://en.wikipedia.org/wiki/Decision_tree_learning">CART</a> algorithm aims the lowest impurty when building nodes. It looks for <em>purety</em> when performing splits. The model will always try to find the most discriminative characteristics over the features with regards to the target.The training process ends when the classification score is above a minimal threshold, or when the tree contains a certain amount of node, or reaches a maximal height.
</p>
<img src="https://cdn-images-1.medium.com/max/1600/0*gx6XsQ8X_KEs7R05.png" />
<p>
    The main benefit of the classification trees is their explainability : each and every sample is classified based on simple and <em>human-readable</em> rules : no explicit probabilities or coefficients are involved (as opposed to Logistic and Linear Regressions).
</p>
<p>
Lets build all the features we need and assemble them : numerical, categorical, and computed features.
</p>

In [1]:
#We import the usuals packages and the model from sklearn 
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
from sklearn.preprocessing import LabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from IPython.display import Image  
from sklearn import tree
import pydotplus
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


In [78]:
dataset = pd.read_csv("./customerLifetimeValue.csv", sep=";")
#We take the columns we need for our models and get the underlying matrix
X_numeric = dataset[["price_first_item_purchased",  "since_birth_parsed_days"]].values.copy()
#We also take a categorical variable
X_categorical = dataset[["Country", "gender"]].copy()
#and we create a new feature
dataset["priceByVisited_pages"] = dataset["price_first_item_purchased"] / dataset["pages_visited"]
X_new_feature  = dataset["priceByVisited_pages"].values.reshape((-1, 1))
#We binarize the target, all value greater than a given revenue will become positive (1), other negative(0)
Y = dataset["revenue"].values
Y[Y <= 175] = 0
Y[Y > 175] = 1

<p>The cell above loads the features. The cell below replaces missing values by "unknown" and produces a column for each possible categorical value (remember the notebook n°4).
</p> 
<p>
    In the following cell, you can notice that one <code>LabelBinarizer</code> is needed per feature.
</p>

In [79]:
X_categorical.fillna("unknown", inplace=True)
CountryBinarizer = LabelBinarizer()
genderBinarizer = LabelBinarizer()
binCountries = CountryBinarizer.fit_transform(X_categorical["Country"])
binGenders = genderBinarizer.fit_transform(X_categorical["gender"])

<p>
    At this stage, the X variable contains all the features and the Y all the targets values (revenue >175\$). We will use a tree classifier as a model.
</p>

In [116]:
#All features are concatenated to produce the final X variable
X = np.hstack([X_numeric, binCountries, binGenders, 1-binGenders, X_new_feature])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

<p>
    Above : splitting the dataset in test and train sets <br /> Below : training a model (note how the api is the same for every model).
</p>

In [119]:
model = DecisionTreeClassifier(max_leaf_nodes=8)
model.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=8, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

<p>
    Similarly, testing the model is as simple as :
</p>

In [127]:
Yhat = model.predict(X_test)
print(accuracy_score(Y_test, Yhat))

0.7379253283690144


<p>
    It's a very decent score considering the size of the dataset and the nature of the model. Now, lets see how we can interpret those results and better understand the model we've produced : the previous models we saw used coefficients associated to each feature. The model automatically established correlations and learned the propper parameters. Trees work differently.
</p>
<p>
    A tree classifier will try to find the most relevant splits (or cuts). It looks for discriminative values in our features with regard to the target. The node of the tree can be displayed using <a href="http://webgraphviz.com/">this link</a> : copy-paste the result of the execution of the following cell (it simply produces a .dot code format that describes the tree) :
</p>


In [129]:
featuresNames = ["price_first_item_purchased",  "since_birth_parsed_days"] + list(CountryBinarizer.classes_) + list(genderBinarizer.classes_) + ["priceByVisited_pages"]
dot_data = tree.export_graphviz(treeClassifier ,out_file="./graph.dot", class_names=["Low Revenue", "High Revenue"], feature_names=featuresNames)
! cat graph.dot

<p>
    Now, an ultimate steps lays between you and your goal of becoming an aprentice data scientist :
<br />
    The data science <em>iteration</em> process is the following : exploration, feature engineering, testing. Now it is your turn to go through the whole cycle. Follow the following steps :
    <ul>
        <li>Use the <code>dataset["since_birth_parsed_days"].hist()</code> function to draw the age distribution of the population</li>
        <li>Based on your observations, build a new feature</li>
        <li>Stack your new feature with the previously created <code>X</code> variable to create a new training set</li>
        <li>Create and train a <code>RandomForestClassifier</code></li>
    </ul>
</p>
<p>
    And before you start, a quick word about Random Forest Classifiers.
</p>
<h2>Random Forest Classifiers</h2>
<p>
    The model we've featured previously was seducing in many ways : it was simple, elegant, understandable, and had decent results. Knowing that this model is callled a <em>Tree</em>, you may now imagine what we designate as a <em>Forest</em>. Again the idea is really intuitive ; instead of using one tree classifier, we use an ensemble of tree classifiers and make them <em>vote</em>.
</p>
<img src="https://www.researchgate.net/profile/Evaldas_Vaiciukynas/publication/301638643/figure/fig1/AS:355471899807744@1461762513154/Architecture-of-the-random-forest-model.png" />

<p>
    The intuition behind this new method is to leverage a well known problem of the trees. The simpliest way of improving the score of a tree is to make it more complex, more <em>convoluted</em>. This produces <em>overfitted</em> models and all explainability is lost. The tree becames over specialized in predicting on the test set and creates ultra-specific rules (you can observe this by re-training a more complex tree on the cells above and draw it). To avoid this, a <code>RandomForestClassifier</code> will train multiple simple trees over small subsets of the train set. This way, we avoid overfitting and we keep the single models simple. Sadly, the interpretability is lost as a sigle prediction is the result of a vote over sometimes more than a thousand trees.
</p>
<p>
    Ok ! Now that you know what you're dealing with, you are ready to produce you're first very own model ! Good luck !
  
</p>